### Código Parcial Javiera Carrasco N., Maximiliano Valladares G.

### initialize_comp_gcm

In [54]:
def initialize_comp_gcm():
    import datetime

    parameters = {
        "data": {
            "name": "C:/Universidad/Simulación Estocástica/Proyecto/Code-Machine_Learning-Sicheng/data/Tan_data-2/GCM.txt",  # Nombre del archivo de datos
            "testing": "TestFile.txt",
            "typet": "Normal",
            "typen": "Tumor",
            "numofgene": 16064,
            "normalize": 1,
            "currentiter": 1,
            "generealization": 1,
        },
        "snapshots": {
            "k1": 89,
            "k1multi": 39,
            "tumrs": [100],
            "normrs": [1000],
            "controlRand": False,
        },
        "multilevel": {
            "l": 8,
            "nested": 1,
        },
        "testtype": "sin",
        "sinfrequency": 20,
        "model": "SVM",
        "parallel": {
            "on": 0,
        },
        "svm": {
            "kernal": 1,
        },
        "rf": {
            "numtree": 100,
        },
        "gb": {
            "rus": 1,
        },
        "testdata": {
            "tumrs": 1000,
            "normrs": 1000,
        },
    }
    model = parameters["model"]
    testtype = parameters["testtype"]
    if model == "SVM":
        if testtype == "sin":
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Model-{model}-Test-{testtype}{parameters['sinfrequency']}-"
                f"Kernel-{parameters['svm']['kernal']}{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
        else:
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Kernel-{parameters['svm']['kernal']}"
                f"{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
    elif model == "GB":
        if testtype == "sin":
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Model-{model}-Test-{testtype}{parameters['sinfrequency']}-"
                f"RUS-{parameters['gb']['rus']}{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
        else:
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Model-{model}-RUS-{parameters['gb']['rus']}"
                f"{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
    elif model == "RF":
        if testtype == "sin":
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Model-{model}-Tree-{parameters['rf']['numtree']}-"
                f"Test-{testtype}{parameters['sinfrequency']}{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
        else:
            parameters["dataname"] = (
                f"Tan-GCM-Gene-Nested-{parameters['multilevel']['nested']}-"
                f"Eigen-{parameters['snapshots']['k1multi']}-Normalized-{parameters['data']['normalize']}-"
                f"Levels-0-{parameters['multilevel']['l']}-Tree-{parameters['rf']['numtree']}-"
                f"Model-{model}{datetime.datetime.now().strftime('%Y-%m-%d')}.mat"
            )
    # Establecer número de simulaciones
    if parameters["data"]["generealization"] == 1:
      parameters["data"]["nk"] = len(parameters["snapshots"]["tumrs"]) - 1
    else:
      parameters["data"]["nk"] = 0

    
    if parameters["parallel"]["on"] == 1:
        parameters["parallel"]["numofproc"] = 4 

    return parameters

### snapshots

In [55]:
import numpy as np
from scipy.sparse.linalg import svds

#### snapshotssub

In [56]:
def snapshotssub(data, k):
    if data.shape[1] < k:
        raise ValueError(f"Number of samples ({data.shape[1]}) is less than the requested eigenvalues ({k}).")

    mx = np.mean(data, axis=1, keepdims=True)
    cx = data - mx  

    M = cx.shape[1]

    u_reduced, s, vt = svds(cx.T @ cx / M, k=k)
    s = s[::-1]
    u_reduced = u_reduced[:, ::-1]
    eigenvectors = cx @ u_reduced
    eigenvectors /= np.linalg.norm(eigenvectors, axis=0)

    eigenfunction = eigenvectors.T / np.sqrt(s[:, None])
    eigenfunction = np.vstack((eigenfunction, mx.T))

    return None, s, eigenvectors, eigenfunction, mx


#### snapshotsreal

In [57]:
def snapshotsreal(eigenfunctions, eigenvalues, mx, numreal, parameters):
    if eigenfunctions.size == 0 or eigenvalues.size == 0:
        raise ValueError("Eigenfunctions or eigenvalues are empty. Cannot generate realizations.")

    eigenfunctions_reduced = eigenfunctions[:-1, :]

    random_vars = (np.random.rand(eigenfunctions_reduced.shape[0], numreal) - 0.5) * 2 * np.sqrt(3)

    realizations_reduced = np.diag(np.sqrt(eigenvalues)) @ random_vars

    realizations_full = eigenfunctions_reduced.T @ realizations_reduced

    realizations = mx + realizations_full

    return realizations


#### snapshots1

In [58]:
def snapshots1(data_origin, parameters, methods, numreal):

    if data_origin.size == 0:
        raise ValueError("Input data 'data_origin' is empty.")

    covariance, eigenvalues, eigenvectors, eigenfunction, mx = methods["Multi"]["snapshotssub"](
        data_origin, parameters["snapshots"]["k1"]
    )

    realizations = None
    if parameters["data"]["generealization"] == 1:
        realizations = snapshotsreal(eigenfunction, eigenvalues, mx, numreal, parameters)

    return {
        "snapshots": {
            "covariance": covariance,
            "eigenvalues": eigenvalues,
            "eigenvectors": eigenvectors,
            "eigenfunction": eigenfunction,
            "mx": mx,
            "realizations": realizations,
        }
    }


#### snapshotsgendata2

In [59]:
def snapshotsgendata2(datas, methods, parameters):
    tum_data = datas['rawdata']['tumData']
    norm_data = datas['rawdata']['normData']
    k = parameters['data']['nk']


    print(f"Dimensiones iniciales: TumData {tum_data.shape}, NormData {norm_data.shape}")

    k = parameters["data"]["nk"]
    if k >= len(parameters["snapshots"]["tumrs"]) or k >= len(parameters["snapshots"]["normrs"]):
      raise IndexError(f"Índice k ({k}) fuera de rango. Listas disponibles: tumrs={len(parameters['snapshots']['tumrs'])}, normrs={len(parameters['snapshots']['normrs'])}")


    np.random.seed(10000)
    tum = snapshots1(tum_data, parameters, methods, parameters['snapshots']['tumrs'][k] + parameters['testdata']['tumrs'])
    np.random.seed(0)
    norm = snapshots1(norm_data, parameters, methods, parameters['snapshots']['normrs'][k] + parameters['testdata']['normrs'])

    datas['rawdata']['tumData'] = tum['snapshots']['realizations'][:, :parameters['snapshots']['tumrs'][k]]
    datas['rawdata']['normData'] = norm['snapshots']['realizations'][:, :parameters['snapshots']['normrs'][k]]
    datas['testdata'] = {
        'tumData': tum['snapshots']['realizations'][:, parameters['snapshots']['tumrs'][k]:],
        'normData': norm['snapshots']['realizations'][:, parameters['snapshots']['normrs'][k]:],
        'tumlabel': np.ones(parameters['testdata']['tumrs']),
        'normlabel': np.zeros(parameters['testdata']['normrs']),
    }

    return datas


### read_cancer_data

In [60]:
def standarized2(data):
    std = np.std(data, axis=0, keepdims=True)
    std[std == 0] = 1 
    return (data - np.mean(data, axis=0, keepdims=True)) / std

def datasize(datas, parameters):
    k = parameters['data']['currentiter'] - 1 

    if parameters['data']['generealization'] == 1:
        n = parameters['snapshots']['normrs'][k] 
    else:
        n = datas['rawdata']['normData'].shape[1]

    t = datas['rawdata']['tumData'].shape[1]

    parameters['data']['t'] = t
    parameters['data']['n'] = n
    return parameters



def selectgene(datas, ngene, ndata):
    tum_data = datas['rawdata']['tumData']
    norm_data = datas['rawdata']['normData']

    max_genes = norm_data.shape[0]
    if ngene < max_genes:
        selected_indices = np.random.choice(max_genes, ngene, replace=False)
    else:
        selected_indices = np.arange(max_genes)

    select_tum_data = tum_data[selected_indices, :]
    select_norm_data = norm_data[selected_indices, :ndata]

    datas['rawdata']['select_tumData'] = select_tum_data
    datas['rawdata']['select_normData'] = select_norm_data

    return datas


In [61]:
def read_cancer_data(parameters, methods):
    file_path = parameters["data"]["name"]
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"El archivo '{file_path}' no existe.")

    df = pd.read_csv(file_path)

    tum_data = df.loc[:, df.columns.str.startswith(parameters["data"]["typen"])].to_numpy()
    norm_data = df.loc[:, df.columns.str.startswith(parameters["data"]["typet"])].to_numpy()

    print("Datos de Tumor originales (parcial):", tum_data[:5, :5])
    print("Datos Normales originales (parcial):", norm_data[:5, :5])

    tum_data = methods["all"]["normalizedata"](tum_data.copy())
    norm_data = methods["all"]["normalizedata"](norm_data.copy())

    print("Datos de Tumor normalizados (parcial):", tum_data[:5, :5])
    print("Datos Normales normalizados (parcial):", norm_data[:5, :5])

    datas = {
        "rawdata": {
            "T": df,
            "tumData": tum_data,
            "normData": norm_data,
        }
    }

    if parameters["data"]["generealization"] == 1:
        datas = methods["Multi"]["generateData"](datas, methods, parameters)

    return datas, parameters


In [62]:
import os
import pandas as pd
import numpy as np

In [63]:
parameters = initialize_comp_gcm()
print("Contenido de parameters antes de llamar a read_cancer_data:")
import pprint
pprint.pprint(parameters)

methods = {
    "all": {
        "normalizedata": standarized2,
        #"prepdata": PrepData,
        "SVMmodel": fit_svm_model,
        "SVMpredict": predict_svm,
        "readcancerData": read_cancer_data,
        "Datasize": datasize,
        "selectgene": selectgene,
        "iniresults": initialize_results,
    },
    "Multi": {
        "snapshotssub": snapshotssub,
        "snapshots": snapshots1,
        "generateData": snapshotsgendata2,
        "orthonormal_basis": orthonormal_basis,
        "dsgnmatrix": design_matrix,
        "multilevel": multilevel,
    },
    "SVMonly": {
        "CompSVMonly": CompSVMonly,
        #"Prep": svmprepdata,
    },
}

datas, parameters = read_cancer_data(parameters, methods)

# Verificar
print("Datos de Tumor:", datas["rawdata"]["tumData"].shape)
print("Datos Normales:", datas["rawdata"]["normData"].shape)



Contenido de parameters antes de llamar a read_cancer_data:
{'data': {'currentiter': 1,
          'generealization': 1,
          'name': 'C:/Universidad/Simulación '
                  'Estocástica/Proyecto/Code-Machine_Learning-Sicheng/data/Tan_data-2/GCM.txt',
          'nk': 0,
          'normalize': 1,
          'numofgene': 16064,
          'testing': 'TestFile.txt',
          'typen': 'Tumor',
          'typet': 'Normal'},
 'dataname': 'Tan-GCM-Gene-Nested-1-Eigen-39-Normalized-1-Levels-0-8-Model-SVM-Test-sin20-Kernel-12024-11-21.mat',
 'gb': {'rus': 1},
 'model': 'SVM',
 'multilevel': {'l': 8, 'nested': 1},
 'parallel': {'on': 0},
 'rf': {'numtree': 100},
 'sinfrequency': 20,
 'snapshots': {'controlRand': False,
               'k1': 89,
               'k1multi': 39,
               'normrs': [1000],
               'tumrs': [100]},
 'svm': {'kernal': 1},
 'testdata': {'normrs': 1000, 'tumrs': 1000},
 'testtype': 'sin'}
Datos de Tumor originales (parcial): [[ -64.3  -13.   -33.   -

### Previas CompMulti_syn

In [64]:
import numpy as np

def orthonormal_basis(parameters):
    orig_basis = parameters['Training']['origin']['snapshots']['eigenfunction']
    Q, R = np.linalg.qr(orig_basis.T)
    orthogonal_basis = Q.T
    parameters['Training']['origin']['snapshots']['eigenfunction'] = orthogonal_basis
    return parameters


In [65]:
def design_matrix(methods, parameters):
    M, numofpoints, data, n, x = methods['Multi']['dsgnmatrixsub'](
        parameters['Training']['origin']['snapshots']['eigenfunction']
    )
    parameters['Training']['origin']['polymodel'] = {'M': M}
    parameters['Training']['dsgnmatrix'] = {
        'origin': {
            'indexsetsize': n,
            'data': data,
            'numofpoints': numofpoints,
            'x': x,
        }
    }
    return parameters


In [66]:
def multilevel(methods, parameters):
    datatree, sortdata, multileveltree, ind, datacell, datalevel = methods['Multi']['multilevelsub'](
        parameters['Training']['dsgnmatrix']['origin']['data'],
        parameters['Training']['dsgnmatrix']['origin'],
        parameters['Training']['origin']['polymodel']
    )
    parameters['Training']['origin']['multilevel'] = {
        'datatree': datatree,
        'sortdata': sortdata,
        'multileveltree': multileveltree,
        'ind': ind,
        'datacell': datacell,
        'datalevel': datalevel,
        'maxlevel': max(datalevel),
    }
    return parameters


In [67]:
def design_matrix_sub(eigenfunction):
    M = eigenfunction.T  
    numofpoints = M.shape[0]  
    data = np.linspace(0, 1, numofpoints)[:, np.newaxis]  
    n = M.shape[1]  
    x = np.arange(1, numofpoints + 1)  

    return M, numofpoints, data, n, x


In [68]:
def multilevel_sub(data, params, polymodel):
    degree = []

    datatree, sortdata = make_tree(
        data=data,
        split_function=split_kd,
        params=params
    )

    multileveltree, ind, datacell, datalevel = multilevelbasis(
        datatree, sortdata, degree, polymodel
    )

    return datatree, sortdata, multileveltree, ind, datacell, datalevel


In [69]:
import numpy as np

def split_kd(data, params):
    spill = params.get('spill', 0)

    variances = np.var(data, axis=0)
    max_var_index = np.argmax(variances)
    split_dir = np.zeros(data.shape[1])
    split_dir[max_var_index] = 1 

    proj_data = np.dot(data, split_dir)

    threshold = np.median(proj_data)

    if spill > 0:
        lower_bound, upper_bound = np.percentile(proj_data, [(0.5 - spill) * 100, (0.5 + spill) * 100])
        comm_idx = (proj_data > lower_bound) & (proj_data < upper_bound)
    else:
        comm_idx = np.zeros_like(proj_data, dtype=bool)

    left_idxs = (proj_data <= threshold) | comm_idx
    right_idxs = (proj_data > threshold) | comm_idx

    return left_idxs, right_idxs, threshold, split_dir, proj_data


#### previa y multilevel_basis

##### make_tree

In [70]:
def make_tree(data, split_function, params):

    max_depth = params.get("MAX_DEPTH", 15)
    split_fxn_params = params.get("split_fxn_params", {"spill": 0})
    indexsetsize = params["indexsetsize"]

    node = 0

    tree, node, data = create_tree(
        data, np.arange(data.shape[0]), split_function, indexsetsize,
        split_fxn_params, max_depth, curr_depth=1, node=node
    )
    tree["size"] = node

    return tree, data


def create_tree(data, idxs, split_function, indexsetsize, split_fxn_params, max_depth, curr_depth, node):

    tree = {
        "idxs": idxs,
        "left": None,
        "right": None,
        "threshold": None,
        "split_dir": None,
        "proj_data": None,
        "center": np.mean(data[idxs, :], axis=0),
        "currentdepth": curr_depth - 1,
        "node": node
    }
    parent_node = node
    node += 1

    if curr_depth >= max_depth or len(idxs) <= (indexsetsize + 1):
        return tree, node, data

    idx_left, idx_right, threshold, split_dir, proj_data = split_function(data[idxs, :], split_fxn_params)

    if len(idx_left) >= indexsetsize and len(idx_right) >= indexsetsize:
        left_idxs = idxs[idx_left]
        right_idxs = idxs[idx_right]

        tree["childleft"] = node
        tree["left"], node, data = create_tree(data, left_idxs, split_function, indexsetsize, split_fxn_params, max_depth, curr_depth + 1, node)

        tree["childright"] = node
        tree["right"], node, data = create_tree(data, right_idxs, split_function, indexsetsize, split_fxn_params, max_depth, curr_depth + 1, node)

        tree["threshold"] = threshold
        tree["split_dir"] = split_dir
        tree["proj_data"] = proj_data

        tree["left"]["parentnode"] = parent_node
        tree["right"]["parentnode"] = parent_node

    return tree, node, data


##### convert_bintree_to_list

In [71]:
def convert_bintree_to_list(tree):

    sizeoftree = tree['size']

    leftchild = [0] * sizeoftree
    rightchild = [0] * sizeoftree
    parent = [0] * sizeoftree
    datalevel = [0] * sizeoftree
    datacell = [None] * sizeoftree

    node = 0

    def createdata(leftchild, rightchild, parent, datacell, tree, datalevel, node):

        datacell[node] = tree.get('idxs', [])
        datalevel[node] = tree.get('currentdepth', 0)

        if 'left' in tree and tree['left'] is not None:
            leftchild[node] = tree.get('childleft', 0)
        if 'right' in tree and tree['right'] is not None:
            rightchild[node] = tree.get('childright', 0)

        if node > 0:
            parent[node] = tree.get('parentnode', 0)

        node += 1

        if 'left' in tree and tree['left'] is not None:
            leftchild, rightchild, parent, datacell, datalevel, node = createdata(
                leftchild, rightchild, parent, datacell, tree['left'], datalevel, node
            )
        if 'right' in tree and tree['right'] is not None:
            leftchild, rightchild, parent, datacell, datalevel, node = createdata(
                leftchild, rightchild, parent, datacell, tree['right'], datalevel, node
            )

        return leftchild, rightchild, parent, datacell, datalevel, node

    leftchild, rightchild, parent, datacell, datalevel, _ = createdata(
        leftchild, rightchild, parent, datacell, tree, datalevel, node
    )

    return leftchild, rightchild, parent, datacell, datalevel


In [72]:
def convert_bintree_to_list(tree):

    if 'size' not in tree or not isinstance(tree['size'], int):
        raise ValueError("El árbol debe contener la clave 'size' con un valor entero.")

    sizeoftree = tree['size']

    leftchild = [0] * sizeoftree
    rightchild = [0] * sizeoftree
    parent = [0] * sizeoftree
    datalevel = [0] * sizeoftree
    datacell = [None] * sizeoftree

    node = 0

    def createdata(leftchild, rightchild, parent, datacell, tree, datalevel, node):

        print(f"Procesando nodo {node}:")
        print(f"  Tamaño de idxs = {len(tree.get('idxs', []))}")
        print(f"  Nivel actual = {tree.get('currentdepth', 0)}")

        datacell[node] = tree.get('idxs', [])
        datalevel[node] = tree.get('currentdepth', 0)

        if 'left' in tree and tree['left'] is not None:
            leftchild[node] = node + 1
            leftchild, rightchild, parent, datacell, datalevel, node = createdata(
                leftchild, rightchild, parent, datacell, tree['left'], datalevel, node + 1
            )
        else:
            leftchild[node] = 0

        if 'right' in tree and tree['right'] is not None:
            rightchild[node] = node + 1
            leftchild, rightchild, parent, datacell, datalevel, node = createdata(
                leftchild, rightchild, parent, datacell, tree['right'], datalevel, node + 1
            )
        else:
            rightchild[node] = 0

        return leftchild, rightchild, parent, datacell, datalevel, node

    leftchild, rightchild, parent, datacell, datalevel, _ = createdata(
        leftchild, rightchild, parent, datacell, tree, datalevel, node
    )

    print("Transformación del árbol a listas completada.")
    print(f"Total de nodos procesados: {sizeoftree}")

    return leftchild, rightchild, parent, datacell, datalevel


##### PolynomialMonomials

In [73]:
import numpy as np

def PolynomialMonomials(degree, coords, polymodel, datapoints):
    
    if 'M' in polymodel:
        return polymodel['M'][datapoints, :]
    else:
        n, dim = coords.shape
        index_set = polymodel['indexset']
        Q = []

        for row in index_set:
            B = np.tile(row, (coords.shape[0], 1))
            C = coords ** B
            Q.append(np.prod(C, axis=1))

        return np.column_stack(Q)


#####computewave

In [74]:
import numpy as np

def computewave(M, V, n):

    U, D, G = np.linalg.svd(M, full_matrices=False)

    numofscalingfunctions = np.count_nonzero(D)

    Cwave = G.T[:, :numofscalingfunctions]
    Scfun = V @ Cwave

    if n > numofscalingfunctions:
        numofwavelets = n - numofscalingfunctions
        Dwave = G.T[:, numofscalingfunctions:numofscalingfunctions + numofwavelets]
        Wavefun = V @ Dwave
    else:
        Wavefun = np.array([])
        Dwave = np.array([])

    return Scfun, Wavefun, Cwave, Dwave


##### leaflocalbasis

In [75]:
import numpy as np

def leaflocalbasis(datapoints, coords, polymodel, degree):

    print(f"Procesando nodo hoja con {len(datapoints)} puntos.")

    filtered_coords = coords[datapoints, :]
    n, m = filtered_coords.shape
    print(f"Dimensiones de coords filtradas: {filtered_coords.shape}")

    V = np.eye(n)

    Q = PolynomialMonomials(degree, filtered_coords, polymodel, datapoints)
    M = Q.T @ V

    Scfun, Wavefun, Cwave, Dwave = computewave(M, V, n)

    return Scfun, Wavefun, Cwave, Dwave


In [76]:
import numpy as np

def leaflocalbasis(datapoints, coords, polymodel, degree):
    
    if not isinstance(datapoints, (list, np.ndarray)):
        raise ValueError("`datapoints` debe ser una lista o un array numpy.")
    if len(datapoints) == 0:
        print("El nodo hoja no contiene puntos, devolviendo None para la base.")
        return None, None, None, None

    if not isinstance(coords, np.ndarray) or coords.ndim != 2:
        raise ValueError("`coords` debe ser un array numpy bidimensional.")

    print(f"Procesando nodo hoja con {len(datapoints)} puntos.")
    filtered_coords = coords[datapoints, :]
    n, m = filtered_coords.shape
    print(f"Dimensiones de coords filtradas: {filtered_coords.shape}")
    print(f"Rango de coordenadas filtradas: {filtered_coords.min(axis=0)} a {filtered_coords.max(axis=0)}")

    V = np.eye(n)

    Q = PolynomialMonomials(degree, filtered_coords, polymodel, datapoints)
    print(f"Dimensiones de Q.T: {Q.T.shape}")
    M = Q.T @ V
    print(f"Dimensiones de M: {M.shape}")

    Scfun, Wavefun, Cwave, Dwave = computewave(M, V, n)

    print("Base jerárquica calculada correctamente.")
    return Scfun, Wavefun, Cwave, Dwave


##### localbasis

In [77]:
def localbasis(Vleft, left_idxs, Vright, right_idxs, idxs, coords, polymodel, degree):
    if Vleft is None:
        Vleft = np.zeros((0, Vright.shape[1]))
    if Vright is None:
        Vright = np.zeros((0, Vleft.shape[1]))

    V = np.block([
        [Vleft, np.zeros((Vleft.shape[0], Vright.shape[1]))],
        [np.zeros((Vright.shape[0], Vleft.shape[1])), Vright]
    ])

    filtered_coords = coords[idxs, :]
    Q = PolynomialMonomials(degree, filtered_coords, polymodel, idxs)
    if Q.T.shape[1] != V.shape[0]:
      raise ValueError(f"Incompatibilidad de dimensiones: Q.T tiene {Q.T.shape[1]} columnas, pero V tiene {V.shape[0]} filas.")

    M = Q.T @ V

    Scfun, Wavefun, Cwave, Dwave = computewave(M, V, len(idxs))

    return Scfun, Wavefun, Cwave, Dwave



In [78]:
def localbasis(Vleft, left_idxs, Vright, right_idxs, idxs, coords, polymodel, degree):
    
    print(f"Dimensiones iniciales en localbasis:")
    print(f"  Vleft: {'None' if Vleft is None else Vleft.shape}, left_idxs: {len(left_idxs)}")
    print(f"  Vright: {'None' if Vright is None else Vright.shape}, right_idxs: {len(right_idxs)}")
    print(f"  idxs: {len(idxs)}")

   
    if Vleft is None:
        Vleft = np.zeros((0, Vright.shape[1]))
        print("Vleft es None, inicializado como matriz vacía.")
    if Vright is None:
        Vright = np.zeros((0, Vleft.shape[1]))
        print("Vright es None, inicializado como matriz vacía.")

    
    if Vleft.shape[0] != len(left_idxs) or Vright.shape[0] != len(right_idxs):
        raise ValueError(
            f"Incompatibilidad: Vleft tiene {Vleft.shape[0]} filas, pero left_idxs tiene {len(left_idxs)} elementos. "
            f"Vright tiene {Vright.shape[0]} filas, pero right_idxs tiene {len(right_idxs)} elementos."
        )

    V = np.block([
        [Vleft, np.zeros((Vleft.shape[0], Vright.shape[1]))],
        [np.zeros((Vright.shape[0], Vleft.shape[1])), Vright]
    ])

    combined_idxs = np.concatenate((left_idxs, right_idxs))
    sorted_key = np.argsort(combined_idxs)
    V = V[sorted_key, :]
    print(f"Dimensiones de V combinada: {V.shape}")

    filtered_coords = coords[idxs, :]
    print(f"Dimensiones de coords filtradas: {filtered_coords.shape}")

    Q = PolynomialMonomials(degree, filtered_coords, polymodel, idxs)
    print(f"Dimensiones de Q.T: {Q.T.shape}")

    if Q.T.shape[1] != V.shape[0]:
        raise ValueError(
            f"Incompatibilidad de dimensiones: Q.T tiene {Q.T.shape[1]} columnas, pero V tiene {V.shape[0]} filas."
        )

    M = Q.T @ V
    print(f"Dimensiones de M: {M.shape}")

    Scfun, Wavefun, Cwave, Dwave = computewave(M, V, len(idxs))

    return Scfun, Wavefun, Cwave, Dwave


##### multilevelbasis

In [79]:
import numpy as np

def multilevelbasis(tree, coords, degree, polymodel):

    leftchild, rightchild, parent, datacell, datalevel = convert_bintree_to_list(tree)

    sortdatalevel = np.sort(datalevel)
    ind = np.argsort(datalevel)
    unsortkey = np.argsort(ind)

    currentlevel = sortdatalevel[-1]
    levelcounter = len(sortdatalevel) - 1

    transformcell = [None] * len(ind)
    for i in range(len(ind)):
        transformcell[i] = [None] * 6

    while sortdatalevel[levelcounter] == currentlevel:
        leaf_data = datacell[ind[levelcounter]]
        transformcell[levelcounter][0], transformcell[levelcounter][1], \
        transformcell[levelcounter][2], transformcell[levelcounter][3] = \
            leaflocalbasis(leaf_data, coords, polymodel, degree)
        levelcounter -= 1

    minlevel = 1

    while currentlevel > minlevel:
        currentlevel -= 1
        while levelcounter >= 0 and sortdatalevel[levelcounter] == currentlevel:
            node_index = ind[levelcounter]

            if leftchild[node_index] == 0 and rightchild[node_index] == 0:
                leaf_data = datacell[node_index]
                transformcell[levelcounter][0], transformcell[levelcounter][1], \
                transformcell[levelcounter][2], transformcell[levelcounter][3] = \
                    leaflocalbasis(leaf_data, coords, polymodel, degree)

            elif leftchild[node_index] != 0 and rightchild[node_index] == 0:
                left_trans = transformcell[unsortkey[leftchild[node_index] - 1]]
                transformcell[levelcounter][0] = left_trans[0]
                transformcell[levelcounter][1] = None
                transformcell[levelcounter][2] = left_trans[2]
                transformcell[levelcounter][3] = None

            elif leftchild[node_index] == 0 and rightchild[node_index] != 0:
                right_trans = transformcell[unsortkey[rightchild[node_index] - 1]]
                transformcell[levelcounter][0] = right_trans[0]
                transformcell[levelcounter][1] = None
                transformcell[levelcounter][2] = right_trans[2]
                transformcell[levelcounter][3] = None

            else:
                left_trans = transformcell[unsortkey[leftchild[node_index] - 1]]
                right_trans = transformcell[unsortkey[rightchild[node_index] - 1]]

                print(f"Procesando nodo {node_index} con dos hijos:")
                print(f"  Vleft dimensiones: {'None' if left_trans[0] is None else left_trans[0].shape}")
                print(f"  Vright dimensiones: {'None' if right_trans[0] is None else right_trans[0].shape}")


                transformcell[levelcounter][0], transformcell[levelcounter][1], \
                transformcell[levelcounter][2], transformcell[levelcounter][3] = \
                    localbasis(
                        left_trans[0], datacell[leftchild[node_index] - 1],
                        right_trans[0], datacell[rightchild[node_index] - 1],
                        datacell[node_index], coords, polymodel, degree
                    )

            levelcounter -= 1

    left_trans = transformcell[unsortkey[leftchild[ind[levelcounter]] - 1]]
    right_trans = transformcell[unsortkey[rightchild[ind[levelcounter]] - 1]]
    transformcell[levelcounter][0], transformcell[levelcounter][1], \
    transformcell[levelcounter][2], transformcell[levelcounter][3] = \
        localbasis(
            left_trans[0], datacell[leftchild[ind[levelcounter]] - 1],
            right_trans[0], datacell[rightchild[ind[levelcounter]] - 1],
            datacell[ind[levelcounter]], coords, polymodel, degree
        )

    for n in range(len(transformcell)):
        transformcell[n][4] = sortdatalevel[n]

    counterscaling, counterwave = 0, 0
    for n in range(len(transformcell) - 1, -1, -1):
        counterscaling += transformcell[n][0].shape[1] if transformcell[n][0] is not None else 0
        counterwave += transformcell[n][1].shape[1] if transformcell[n][1] is not None else 0
        transformcell[n][5] = counterscaling
        transformcell[n].append(counterwave)

    return transformcell, ind, datacell, datalevel


### CompMulti_Syn

In [80]:
def comp_multi_syn(methods, datas, parameters, results):

    t = parameters['data'].get('t', None)
    n = parameters['data'].get('n', None)

    if t is None or n is None:
        raise ValueError("Las variables 't' o 'n' no están definidas en 'parameters'.")
    if n > datas['rawdata']['normData'].shape[1]:
        raise ValueError(f"'n' ({n}) es mayor que el número de columnas en 'normData' ({datas['rawdata']['normData'].shape[1]}).")

    datas['ML'] = {}
    datas['ML']['Training'] = datas['rawdata']['normData'][:, t:n]
    print(f"Dimensiones de Training: {datas['ML']['Training'].shape}")

    if datas['ML']['Training'].size == 0:
        raise ValueError("Los datos de entrenamiento ('Training') están vacíos. Verifica las configuraciones de 't' y 'n'.")

    parameters['Training'] = {}
    parameters['Training']['origin'] = {}
    snapshots = methods['Multi']['snapshots'](datas['ML']['Training'], parameters, methods, n)

    parameters['Training']['origin']['snapshots'] = snapshots['snapshots']

    print("Claves en snapshots:", parameters['Training']['origin']['snapshots'].keys())

    if 'eigenfunction' not in parameters['Training']['origin']['snapshots']:
        raise KeyError("La clave 'eigenfunction' no está presente en snapshots. Verifica la función snapshots.")

    parameters = methods['Multi']['orthonormal_basis'](parameters)
    parameters = methods['Multi']['dsgnmatrix'](methods, parameters)
    parameters = methods['Multi']['multilevel'](methods, parameters)

    if parameters['parallel']['on'] == 1:
        results = methods['Multi']['parallel'](methods, datas, parameters, results)
    else:
        results = methods['Multi']['noparallel'](methods, datas, parameters, results)

    return results



In [81]:
methods = {
    "all": {
        "normalizedata": standarized2
    },
    "Multi": {
        "snapshotssub": snapshotssub,          
        "snapshots": snapshots1,               
        "generateData": snapshotsgendata2,     
        "orthonormal_basis": orthonormal_basis,
        "dsgnmatrix": design_matrix,
        "dsgnmatrixsub": design_matrix_sub,
        "multilevel": multilevel,               
        "multilevelsub": multilevel_sub,        
    }
}


In [82]:
parameters = initialize_comp_gcm()
datas, parameters = read_cancer_data(parameters, methods)

parameters = datasize(datas, parameters)
print(f"t: {parameters['data']['t']}, n: {parameters['data']['n']}")

datas = selectgene(datas, parameters['data']['numofgene'], parameters['data']['n'])

results = {}
results = comp_multi_syn(methods, datas, parameters, results)

print("Resultados:", results)



Datos de Tumor originales (parcial): [[ -64.3  -13.   -33.   -44.   -18. ]
 [-184.9 -124.   -39.  -254.    28. ]
 [-334.4 -206.   -45.  -166.    10. ]
 [-102.2  -29.    14.   -55.    30. ]
 [-289.  -117.   -56.  -130.   -56. ]]
Datos Normales originales (parcial): [[  12.    61.    -5.   -13.3  -48. ]
 [-231.  -580.  -608.  -344.5  -86. ]
 [-287.  -980.  -810.  -489.7  -51. ]
 [  12.    76.   163.    23.8  -21. ]
 [-257.  -444.  -429.  -274.8  -60. ]]
Datos de Tumor normalizados (parcial): [[-0.28068351 -0.10516373 -0.32715033 -0.23824139 -0.21535171]
 [-0.58721029 -0.48238056 -0.3369614  -1.05937955 -0.11711991]
 [-0.96719168 -0.76104526 -0.34677248 -0.71528356 -0.15555844]
 [-0.37701324 -0.15953733 -0.25029689 -0.28125339 -0.11284896]
 [-0.85179934 -0.45859212 -0.36475946 -0.57451702 -0.29649972]]
Datos Normales normalizados (parcial): [[-0.33211896 -0.32298974 -0.37610121 -0.26725581 -0.31212631]
 [-0.56496735 -0.74822762 -0.78700859 -0.60312121 -0.42221595]
 [-0.61862788 -1.0135866

ValueError: Incompatibilidad: Vleft tiene 0 filas, pero left_idxs tiene 16063 elementos. Vright tiene 62 filas, pero right_idxs tiene 62 elementos.

### Aquí va DataSVM

In [83]:
def datasvm(datas, parameters, methods, l):

    tX_Train, ty_Train = methods['Multi']['datasvmsub1'](parameters['Training']['tumor']['C'], l)
    nX_Train, ny_Train = methods['Multi']['datasvmsub2'](parameters['Training']['normal']['C'], l)

    X_Train = np.vstack((tX_Train, nX_Train)) 
    y_Train = np.hstack((ty_Train, ny_Train)) 

    datas['X_Train'] = X_Train
    datas['y_Train'] = y_Train
    datas['X_Train_tumor'] = tX_Train
    datas['X_Train_normal'] = nX_Train

    tX_Test, ty_Test = methods['Multi']['datasvmsub1'](parameters['Testing']['tumor']['C'], l)
    nX_Test, ny_Test = methods['Multi']['datasvmsub2'](parameters['Testing']['normal']['C'], l)

    X_Test = np.vstack((tX_Test, nX_Test))
    y_Test = np.hstack((ty_Test, ny_Test))

    datas['X_Test'] = X_Test
    datas['y_Test'] = y_Test
    datas['X_Test_tumor'] = tX_Test
    datas['X_Test_normal'] = nX_Test

    return datas

### Acá va CompMultiNoParallel_Syn Y OJO CON HBTRANS

In [84]:
def hbtrans(fdata, transformcell, ind, datacell, datalevel):

    numofnodes = len(transformcell)
    coeff = []
    levelcoeff = []
    dcoeffs = [None] * numofnodes
    ccoeffs = None

    for n in range(numofnodes - 1, -1, -1):
        if transformcell[n][1] is not None:
            ixds = datacell[ind[n]]
            W = transformcell[n][1]  
            localcoeff = fdata[ixds].T @ W  
            coeff.extend(localcoeff)  
            levelcoeff.extend([datalevel[ind[n]]] * W.shape[1]) 
            dcoeffs[n] = localcoeff

    if transformcell[0][0] is not None:
        V = transformcell[0][0]  
        ccoeffs = fdata.T @ V  
        coeff.extend(ccoeffs)  
        levelcoeff.extend([-1] * V.shape[1])  
        coeff = np.array(coeff).T  

    nfdata = len(fdata)
    outputcoeff = np.zeros(nfdata)
    outputcoeff[:len(coeff)] = coeff

    return outputcoeff, np.array(levelcoeff), dcoeffs, ccoeffs


In [85]:
def Coeffhbtrans(m, C, Q, multileveltree, inds, datacell, datalevel):
    
    for i in range(m):
        hQ = Q[:, i]
        coeff, levelcoeff, dcoeffs, ccoeffs = hbtrans(hQ, multileveltree, inds, datacell, datalevel)
        C[:, i] = coeff
        C[:, m] = levelcoeff

    return C, levelcoeff, dcoeffs, ccoeffs


In [86]:
def nesteddatasvmsub1(C, l):

    m = C.shape[1]

    last_column = C[:, -1]
    X = C[last_column <= l, :]

    last_column = X[:, -1]
    X = X[last_column >= 0, :]

    X = X[:, :-1]

    y = np.ones((m - 1,))

    X = X.T
    y = y.T

    return X, y


In [87]:
def nesteddatasvmsub2(C, l):

    m = C.shape[1]

    last_column = C[:, -1]
    X = C[last_column <= l, :]

    last_column = X[:, -1]
    X = X[last_column >= 0, :]


    X = X[:, :-1]

    y = np.zeros((m - 1,))

    X = X.T
    y = y.T

    return X, y


In [88]:
def GetCoeff(datas, parameters):
    
    tnum = datas['tumor']['Training'].shape[1]
    nnum = datas['normal']['Training'].shape[1]

    tC = np.zeros((parameters['Training']['dsgnmatrix']['origin']['numofpoints'], tnum + 1))
    nC = np.zeros((parameters['Training']['dsgnmatrix']['origin']['numofpoints'], nnum + 1))

    tC, tlevelcoeff, tdcoeffs, tccoeffs = methods["Multi"]["Coeffhbtrans"](
        tnum, tC,
        datas['tumor']['Training'],
        parameters['Training']['origin']['multilevel']['multileveltree'],
        parameters['Training']['origin']['multilevel']['ind'],
        parameters['Training']['origin']['multilevel']['datacell'],
        parameters['Training']['origin']['multilevel']['datalevel']
    )

    nC, nlevelcoeff, ndcoeffs, nccoeffs = methods["Multi"]["Coeffhbtrans"](
        nnum, nC,
        datas['normal']['Training'],
        parameters['Training']['origin']['multilevel']['multileveltree'],
        parameters['Training']['origin']['multilevel']['ind'],
        parameters['Training']['origin']['multilevel']['datacell'],
        parameters['Training']['origin']['multilevel']['datalevel']
    )

    parameters['Training']['tumor']['C'] = tC
    parameters['Training']['normal']['C'] = nC
    parameters['Training']['tumor']['levelcoeff'] = tlevelcoeff
    parameters['Training']['normal']['levelcoeff'] = nlevelcoeff

    
    tnum = datas['tumor']['Testing'].shape[1]
    nnum = datas['normal']['Testing'].shape[1]

    tC = np.zeros((parameters['Training']['dsgnmatrix']['origin']['numofpoints'], tnum + 1))
    nC = np.zeros((parameters['Training']['dsgnmatrix']['origin']['numofpoints'], nnum + 1))

    tC, tlevelcoeff, tdcoeffs, tccoeffs = methods["Multi"]["Coeffhbtrans"](
        tnum, tC,
        datas['tumor']['Testing'],
        parameters['Training']['origin']['multilevel']['multileveltree'],
        parameters['Training']['origin']['multilevel']['ind'],
        parameters['Training']['origin']['multilevel']['datacell'],
        parameters['Training']['origin']['multilevel']['datalevel']
    )

    nC, nlevelcoeff, ndcoeffs, nccoeffs = methods["Multi"]["Coeffhbtrans"](
        nnum, nC,
        datas['normal']['Testing'],
        parameters['Training']['origin']['multilevel']['multileveltree'],
        parameters['Training']['origin']['multilevel']['ind'],
        parameters['Training']['origin']['multilevel']['datacell'],
        parameters['Training']['origin']['multilevel']['datalevel']
    )

    parameters['Testing']['tumor']['C'] = tC
    parameters['Testing']['normal']['C'] = nC
    parameters['Testing']['tumor']['levelcoeff'] = tlevelcoeff
    parameters['Testing']['normal']['levelcoeff'] = nlevelcoeff

    return parameters


In [89]:
def nesteddatasvm(datas, parameters, methods, l):
    
    tX_Train, ty_Train = methods["Multi"]["nesteddatasvmsub1"](parameters['Training']['tumor']['C'], l)
    nX_Train, ny_Train = methods["Multi"]["nesteddatasvmsub2"](parameters['Training']['normal']['C'], l)

    X_Train = np.vstack((tX_Train, nX_Train)) 
    y_Train = np.hstack((ty_Train, ny_Train))

    datas['X_Train'] = X_Train
    datas['y_Train'] = y_Train
    datas['X_Train_tumor'] = tX_Train
    datas['X_Train_normal'] = nX_Train

    tX_Test, ty_Test = methods["Multi"]["nesteddatasvmsub1"](parameters['Testing']['tumor']['C'], l)
    nX_Test, ny_Test = methods["Multi"]["nesteddatasvmsub2"](parameters['Testing']['normal']['C'], l)

    X_Test = np.vstack((tX_Test, nX_Test))
    y_Test = np.hstack((ty_Test, ny_Test))

    datas['X_Test'] = X_Test
    datas['y_Test'] = y_Test
    datas['X_Test_tumor'] = tX_Test
    datas['X_Test_normal'] = nX_Test

    return datas


In [91]:
def CompMultiNoParallel_Syn(methods, datas, parameters, results):

    datas['tumor']['Training'] = datas['rawdata']['select_tumData']
    datas['normal']['Training'] = datas['rawdata']['select_normData']
    datas['tumor']['Testing'] = datas['testdata']['tumData']
    datas['normal']['Testing'] = datas['testdata']['normData']

    all_data = np.concatenate(
        (
            datas['tumor']['Training'],
            datas['normal']['Training'],
            datas['tumor']['Testing'],
            datas['normal']['Testing'],
        ),
        axis=1,
    ).T
    unique_data = np.unique(all_data, axis=0)
    repeated_samples = all_data.shape[0] - unique_data.shape[0]
    print(
        f"All Data size {all_data.shape[0]}, Unique Data size {unique_data.shape[0]}, Non-unique samples {repeated_samples}"
    )
    if repeated_samples != 0:
        print("Warning: Repeated samples, possible data leakage")

    parameters = methods["Multi"]["Getcoeff"](datas, parameters)
    
    for l in range(parameters["multilevel"]["l"] + 1):
        datas2 = datas.copy()
        parameters2 = parameters.copy()

        if parameters["multilevel"]["nested"] == 1:
            datas2 = methods["Multi"]["nesteddatasvm"](datas2, parameters2, methods, l)
        else:
            datas2 = methods["Multi"]["datasvm"](datas2, parameters2, methods, l)

        if parameters2["svm"]["kernal"] == 1:
            parameters2["multilevel"]["SVMModel"] = methods["all"]["SVMmodel"](
                datas2["X_Train"],
                datas2["y_Train"],
                KernelFunction="RBF",
                KernelScale="auto",
            )
        else:
            parameters2["multilevel"]["SVMModel"] = methods["all"]["SVMmodel"](
                datas2["X_Train"], datas2["y_Train"]
            )

        y_Test_tumor = methods["all"]["SVMpredict"](
            parameters2["multilevel"]["SVMModel"], datas2["X_Test_tumor"]
        )
        y_Test_normal = methods["all"]["SVMpredict"](
            parameters2["multilevel"]["SVMModel"], datas2["X_Test_normal"]
        )

        err, TP, FP = 0, 0, 0
        for i in range(parameters["testdata"]["tumrs"]):
            if y_Test_tumor[i] != datas["testdata"]["tumlabel"][i]:
                err += 1
            else:
                TP += 1

        for i in range(parameters["testdata"]["normrs"]):
            if y_Test_normal[i] != datas["testdata"]["normlabel"][i]:
                err += 1
                FP += 1

        accuracy = 1 - (err / (parameters["testdata"]["tumrs"] + parameters["testdata"]["normrs"]))
        precision = TP / parameters["testdata"]["tumrs"]

        results["multilevel"]["accuracy"][parameters["data"]["currentiter"], l + 1] = accuracy
        results["multilevel"]["precision"][parameters["data"]["currentiter"], l + 1] = precision

    return results


### Acá irá CompSVMOnly

In [92]:
from joblib import Parallel, delayed

def CompSVMonlyParallel(methods, datas, parameters, results):

    def process_sample(j):
        correct_t, correct_n, wrong_t, wrong_n = 0, 0, 0, 0
        datas2 = datas.copy()
        parameters2 = parameters.copy()

        for i in range(parameters["data"]["t"]):
            parameters2["data"]["i"] = i
            parameters2["data"]["j"] = j

            datas2 = methods["all"]["prepdata"](datas, parameters2)
            datas2 = methods["SVMonly"]["Prep"](datas2)

            if parameters2["svm"]["kernal"] == 1:
                parameters2["svm"]["SVMModel"] = methods["all"]["SVMmodel"](
                    datas2["X_Train"], datas2["y_Train"], KernelFunction="RBF", KernelScale="auto"
                )
            else:
                parameters2["svm"]["SVMModel"] = methods["all"]["SVMmodel"](datas2["X_Train"], datas2["y_Train"])

            y_test_tumor = methods["all"]["SVMpredict"](parameters2["svm"]["SVMModel"], datas2["tumor"]["Testing"].T)
            y_test_normal = methods["all"]["SVMpredict"](parameters2["svm"]["SVMModel"], datas2["normal"]["Testing"].T)

            if y_test_normal == 0:
                correct_n += 1
            elif y_test_normal == 1:
                wrong_n += 1

            if y_test_tumor == 1:
                correct_t += 1
            elif y_test_tumor == 0:
                wrong_t += 1

        return correct_t, correct_n, wrong_t, wrong_n

    results_list = Parallel(n_jobs=-1)(delayed(process_sample)(j) for j in range(parameters["data"]["n"]))

    correct_t = sum(r[0] for r in results_list)
    correct_n = sum(r[1] for r in results_list)
    wrong_t = sum(r[2] for r in results_list)
    wrong_n = sum(r[3] for r in results_list)

    results["svm"]["accuracy"][parameters["data"]["currentiter"], 0] = (correct_t + correct_n) / (
        correct_t + wrong_t + correct_n + wrong_n
    )
    results["svm"]["precision"][parameters["data"]["currentiter"], 0] = correct_t / (correct_t + wrong_n)

    return results


In [93]:
def CompSVMonlyNoParallel(methods, datas, parameters, results):

    correct_t, correct_n, wrong_t, wrong_n = 0, 0, 0, 0

    for j in range(parameters["data"]["n"]):
        datas2 = datas.copy()
        parameters2 = parameters.copy()

        for i in range(parameters["data"]["t"]):
            parameters2["data"]["i"] = i
            parameters2["data"]["j"] = j

            datas2 = methods["all"]["prepdata"](datas, parameters2)
            datas2 = methods["SVMonly"]["Prep"](datas2)

            if parameters2["svm"]["kernal"] == 1:
                parameters2["svm"]["SVMModel"] = methods["all"]["SVMmodel"](
                    datas2["X_Train"], datas2["y_Train"], KernelFunction="RBF", KernelScale="auto"
                )
            else:
                parameters2["svm"]["SVMModel"] = methods["all"]["SVMmodel"](datas2["X_Train"], datas2["y_Train"])

            y_test_tumor = methods["all"]["SVMpredict"](parameters2["svm"]["SVMModel"], datas2["tumor"]["Testing"].T)
            y_test_normal = methods["all"]["SVMpredict"](parameters2["svm"]["SVMModel"], datas2["normal"]["Testing"].T)

            if y_test_normal == 0:
                correct_n += 1
            elif y_test_normal == 1:
                wrong_n += 1

            if y_test_tumor == 1:
                correct_t += 1
            elif y_test_tumor == 0:
                wrong_t += 1

    results["svm"]["accuracy"][parameters["data"]["currentiter"], 0] = (correct_t + correct_n) / (
        correct_t + wrong_t + correct_n + wrong_n
    )
    results["svm"]["precision"][parameters["data"]["currentiter"], 0] = correct_t / (correct_t + wrong_n)

    return results


In [94]:
def CompSVMonly(methods, datas, parameters, results):
    
    if parameters["parallel"]["on"] == 1:
        results = methods["SVMonly"]["parallel"](methods, datas, parameters, results)
    else:
        results = methods["SVMonly"]["noparallel"](methods, datas, parameters, results)

    return results


In [95]:
from sklearn.svm import SVC

def fit_svm_model(X_train, y_train, KernelFunction="linear", KernelScale="auto"):
    if KernelFunction == "RBF":
        model = SVC(kernel="rbf", gamma="scale" if KernelScale == "auto" else KernelScale)
    else:
        model = SVC(kernel="linear")
    model.fit(X_train, y_train)
    return model


def predict_svm(model, X_test):
    return model.predict(X_test)


def initialize_results(parameters):
    results = {
        "svm": {
            "accuracy": np.zeros((parameters["data"]["nk"], 1)),
            "precision": np.zeros((parameters["data"]["nk"], 1)),
        }
    }
    return results


In [96]:
parameters = initialize_comp_gcm()

print("Parámetros iniciales:")
import pprint
pprint.pprint(parameters)

methods = {
    "all": {
        "normalizedata": standarized2,
        "prepdata": PrepData,
        "SVMmodel": fit_svm_model,
        "SVMpredict": predict_svm,
        "readcancerData": read_cancer_data,
        "Datasize": datasize,
        "selectgene": selectgene,
        "iniresults": initialize_results,
    },
    "Multi": {
        "snapshotssub": snapshotssub,
        "snapshots": snapshots1,
        "generateData": snapshotsgendata2,
        "orthonormal_basis": orthonormal_basis,
        "dsgnmatrix": design_matrix,
        "multilevel": multilevel,
    },
    "SVMonly": {
        "CompSVMonly": CompSVMonly,
        "Prep": svmprepdata,
    },
}

datas, parameters = methods["all"]["readcancerData"](parameters, methods)

print("Datos cargados:")
print(f"Tumor data: {datas['rawdata']['tumData'].shape}")
print(f"Normal data: {datas['rawdata']['normData'].shape}")

parameters = methods["all"]["Datasize"](datas, parameters)
print(f"t: {parameters['data']['t']}, n: {parameters['data']['n']}")

datas = methods["all"]["selectgene"](datas, parameters['data']['numofgene'], parameters['data']['n'])
print("Genes seleccionados:")
print(f"Dimensiones de select_normData: {datas['rawdata']['select_normData'].shape}")

results = methods["all"]["iniresults"](parameters)

print("Ejecutando CompSVMonly...")
results = methods["SVMonly"]["CompSVMonly"](methods, datas, parameters, results)

print("Resultados de CompSVMonly:")
print(results)


Parámetros iniciales:
{'data': {'currentiter': 1,
          'generealization': 1,
          'name': 'C:/Universidad/Simulación '
                  'Estocástica/Proyecto/Code-Machine_Learning-Sicheng/data/Tan_data-2/GCM.txt',
          'nk': 0,
          'normalize': 1,
          'numofgene': 16064,
          'testing': 'TestFile.txt',
          'typen': 'Tumor',
          'typet': 'Normal'},
 'dataname': 'Tan-GCM-Gene-Nested-1-Eigen-39-Normalized-1-Levels-0-8-Model-SVM-Test-sin20-Kernel-12024-11-21.mat',
 'gb': {'rus': 1},
 'model': 'SVM',
 'multilevel': {'l': 8, 'nested': 1},
 'parallel': {'on': 0},
 'rf': {'numtree': 100},
 'sinfrequency': 20,
 'snapshots': {'controlRand': False,
               'k1': 89,
               'k1multi': 39,
               'normrs': [1000],
               'tumrs': [100]},
 'svm': {'kernal': 1},
 'testdata': {'normrs': 1000, 'tumrs': 1000},
 'testtype': 'sin'}


NameError: name 'PrepData' is not defined

### Acá va una idea de cómo hubiésemos llegado a la implementación a multiclase

1. *Inicialización*

    - Configurar los parámetros iniciales utilizando la función initialize_comp_gcm().
    - Leer los datos desde un archivo utilizando read_cancer_data(parameters, methods).

2. *Preprocesamiento*

    - Normalizar los datos de tumores y normales utilizando la función standarized2.
    - Seleccionar genes relevantes con la función selectgene, utilizando los parámetros configurados.

3. *Construcción de la Base Multinivel*

    - Generar realizaciones de datos mediante snapshotsgendata2.
    - Construir la base multinivel utilizando las funciones:
        - orthonormal_basis para crear una base ortonormal.
        - design_matrix para diseñar la matriz correspondiente.
        - multilevel para estructurar los niveles del modelo.

4. *Entrenamiento del Modelo SVM*

    - Iterar por cada nivel del modelo multinivel:
        - Generar los datos del nivel actual utilizando nesteddatasvm.
        - Entrenar un modelo SVM:
            - Si se selecciona un kernel RBF, usar fit_svm con el parámetro kernel='RBF'.
            - En caso contrario, usar fit_svm con el parámetro kernel='linear'.

5. *Evaluación del Modelo*

    - Realizar predicciones para datos de tumores y normales utilizando predict_svm.
    - Calcular métricas de evaluación como precisión y exactitud con la función evaluate_model.

6. *Almacenamiento y Resultados*

    - Guardar los resultados de precisión y exactitud para cada nivel en la estructura de resultados.
    - Almacenar los resultados finales en un archivo utilizando la función save_results.